In [1]:
%matplotlib inline

import os 
import sys 
import time
import hashlib

from datetime  import datetime 

import numpy as np  # learn 
import pandas as pd # learn
from pandas import *
from numpy import *

from scipy import stats # look at scipy
from scipy import linalg
from scipy import *

 
import matplotlib as mplt # learn matplolib 
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d

import seaborn as sns 
sns.set_style("whitegrid")
sns.set(rc={"figure.figsize": (14, 6)})

import IPython
from IPython.display import display


import matplotlib
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
 
import sklearn as sk
import itertools



from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.types import *

from pyspark.mllib.regression import LabeledPoint
import random
import time
import copy


In [2]:
#BT data
# dta_RDD = sc.textFile("file:///home/tguo/data/tian-test/udpjitter-H1april2014_regTree.csv")

dta_RDD = sc.textFile("hdfs://computer61.ant-net/user/tguo/udpjitter-H1april2014_regTree_mllib.csv")

#data format: dependent variable, feature values
dta_splited = dta_RDD.map(lambda line: line.split(",")).map(lambda r:
                                                                (float(r[0]),int( float(r[1])), 
                                                                 int(float(r[2])),int(float(r[3])),
                                                                  int(float(r[4])),int(float(r[5])),
                                                                 int(float(r[6])),int(float(r[7])) ))


# dta_splited.first()
dta= dta_splited
dta.cache()
print dta.first()
print dta.count()

# dta_filtered = dta.filter(lambda line: line[0] <2000 )
# dta_filtered.cache()
# dta_filtered.count()
# print dta.count()

(328.0, 10, 10, 2, 2, 8, 1, 3)
3315067


In [27]:
# synthetic data
dta_RDD = sc.textFile("file:///home/tguo/tian_src/synData_7f_6zeros_0.01percen.txt")
# synthetic_data_6zeros_0.01percen.txt

dta_RDD.cache()

dta_splited = dta_RDD.map(lambda line: line.split(",")).map(lambda r: ( float(r[7]),int(r[0]),int(r[1]),int(r[2])\
                                                                       ,int(r[3]) )\
                                                           ) 
# ( float(r[2]),float(r[0]),float(r[1]) ) 

dta= dta_splited
dta.cache()

print 'original:',dta.take(1)
print dta.count()

# re-set index of categorical features

# feature_dist=[]

# tmp1 = dta.map(lambda line: line[1]).distinct().collect()
# feature_dist.append(tmp1)

# tmp1 = dta.map(lambda line: line[2]).distinct().collect()
# feature_dist.append(tmp1)

# tmp1 = dta.map(lambda line: line[3]).distinct().collect()
# feature_dist.append(tmp1)

# fea_cnt= len(feature_dist )
# fea_map=[]

# for i in range(0, fea_cnt):
#     tmpcnt = len(feature_dist[i])
#     val_map= dict( zip(feature_dist[i], range(0,tmpcnt)) )
#     fea_map.append(val_map)

# def reset_index( line ):
#     tmp=[]
#     tmp.append(line[0])
#     for i in range(1,4):
#         tmp.append(fea_map[i-1][ line[i] ] )
#     return tmp

# dta = dta.map( lambda line: reset_index(line)  )

# print 'feature value re-indexed:',dta.first()
# print dta.count()



# total_featureVal_set=[]
# for i in range(0, numFeatures):
#     featureValues = dta_train.map(lambda line: line[1+i]).distinct().collect()
#     total_featureVal_set.append( featureValues)

# print total_featureVal_set    
    
# print 'done'


original: [(78.676903536, 5, 13, 101, 1005)]
2000000


In [3]:
# basic functions in one-pass method [on the cluster side]

# assign each data instance to the node
def search_nodeToData(features, tree):
    nodeNum=len(tree)   
    if nodeNum == 0:
        return 0;
    
    current_nodeIdx=0
    while current_nodeIdx< nodeNum:
        split_feature= tree[current_nodeIdx][0]
        split_valueSet= tree[current_nodeIdx][1]
        
        # unbalanced tree grow
        if split_feature==-1:
            current_nodeIdx=current_nodeIdx*2+1+ random.randint(0, 1)
            continue
            
        if (( 1<<features[split_feature]) &  split_valueSet) != 0 :
            current_nodeIdx=current_nodeIdx*2+1
        else:
            current_nodeIdx=current_nodeIdx*2+2
            
    return current_nodeIdx - nodeNum

def partition_dataToNode(list_dvAndfeatures):
    dataToNode_map=[]
    res=[]
    for dvAndfeatures in list_dvAndfeatures:
        
        Y= dvAndfeatures[0]
        node= search_nodeToData(dvAndfeatures[1:numFeatures+1], \
                                currentNode_split_fromMaster.value)
        dataToNode_map.append(node)
        res.append( (node, dvAndfeatures )   )
    return res
def dataToNode_assignment( data_rdd ):
    dataToNode_map = data_rdd.mapPartitions( partition_dataToNode )
    dataToNode_map.cache()
    return dataToNode_map


#--------------------------------------------------------------------------

# extract values for each feature
def partition_featureValues(list_dvAndfeatures):

    feature_valueSet={}
    
    for i in range(0,numFeatures):
        feature_valueSet.update( {i: set()} )
        
    for dvAndfeatures in list_dvAndfeatures:
        for i in range(0,numFeatures):
            feature_val= dvAndfeatures[i+1]
            feature_valueSet[i].add(feature_val)
    return zip( feature_valueSet.keys(), feature_valueSet.values() )  

def merge_featureValues(  valueSet1, valueSet2):
    return valueSet1.union(valueSet2)
    
def data_featureValues_collect( data_rdd ):
    feature_valueSet_part = data_rdd.mapPartitions( partition_featureValues )
    feature_valueSet_local = \
    feature_valueSet_part.reduceByKey(lambda set1, set2: merge_featureValues(set1,set2 )).collect()
    feature_valueSet_local.sort()
    #test
#     print feature_valueSet_local
    return feature_valueSet_local


In [4]:
# split decision [on the local side]

#implementation notes:
# bit-operation to record the set of selected feature values in the left child

#asecending order, upper: larger, lower: smaller
def upper_tail_boundary( percentage, dat_rdd, tt_cnt):
    tmpcnt= int(tt_cnt*percentage) 
    tmplist = dat_rdd.top( int(tt_cnt*percentage)  )
    
    
    return tmplist[tmpcnt-1]

def lower_tail_boundary( percentage, dat_rdd, tt_cnt):
#     print tt_cnt,percentage, type(dat_rdd)
    sorted_rdd = dat_rdd.sortBy(lambda line:line, ascending= True).cache()
    tmpcnt= int(tt_cnt*percentage) 
    tmplist= dat_rdd.take( tmpcnt )
    return tmplist[ tmpcnt-1 ]
    
# def trimmed_MSE_cal(dat_rdd, dat_rdd_cnt, trim_percentage ):
    
#     upper_bound = upper_tail_boundary( trim_percentage, dat_rdd, dat_rdd_cnt)
# #     lower_bound = lower_tail_boundary( trim_percentage, dat_rdd, dat_rdd_cnt)
    
#     trimmed_rdd= dat_rdd.filter( lambda line: line < upper_bound ).cache()
# #     trimmed_rdd= trimmed_rdd.filter(lambda line: line > lower_bound).cache()
#     trimmed_rdd_cnt = trimmed_rdd.count()
    
#     if trimmed_rdd_cnt ==0:
        
#         print 'trimmed zeor happens!'
#         tmp_mean=0
#         tmp_mse=0
        
#     else:
#         tmp_mean = trimmed_rdd.reduce(lambda a,b:a+b) / trimmed_rdd_cnt
#         tmp_mse= trimmed_rdd.map(lambda line:(line-tmp_mean)*(line-tmp_mean)).reduce(lambda a,b:a+b)/trimmed_rdd_cnt
    
#     return (trimmed_rdd_cnt, tmp_mean, tmp_mse, trimmed_rdd)

def trimmed_MSE( dataToSplit_rdd, feature_id, value_set ):
    
    tmp_rdd=dataToSplit_rdd.filter(lambda l:l[1][feature_id+1] in value_set).map(lambda l:l[1][0]).cache()
    tmp_rdd_count = tmp_rdd.count()
    
    if tmp_rdd_count ==0:
        
        print 'trimmed zeor happens!'
        tmp_mean=0
        tmp_mse=0
    else:
        tmp_mean = tmp_rdd.reduce(lambda a,b:a+b)*1.0 / tmp_rdd_count
        tmp_mse= tmp_rdd.map(lambda line:(line-tmp_mean)*(line-tmp_mean)).reduce(lambda a,b:a+b)*1.0/tmp_rdd_count
    
    return (tmp_rdd_count, tmp_mean, tmp_mse )
    
def trimmed_MSE_node( dataToSplit_rdd, trim_percentage ):
    
    tmp_rdd=dataToSplit_rdd.map(lambda l:l[1][0]).cache()
    tmp_rdd_count = tmp_rdd.count()
    
    upper_bound = upper_tail_boundary( trim_percentage, tmp_rdd, tmp_rdd_count)
#     lower_bound = lower_tail_boundary( trim_percentage, dat_rdd, dat_rdd_cnt)
    tmp_trim_rdd= tmp_rdd.filter( lambda line: line < upper_bound ).cache()
 
    trimmed_rdd= dataToSplit_rdd.filter( lambda line: line[1][0] < upper_bound ).cache()
#     trimmed_rdd= trimmed_rdd.filter(lambda line: line > lower_bound).cache()
    trimmed_rdd_cnt = trimmed_rdd.count()
    
    if trimmed_rdd_cnt ==0:
        
        print 'trimmed zeor happens!'
        tmp_mean=0
        tmp_mse=0
        
    else:
        tmp_mean = tmp_trim_rdd.reduce(lambda a,b:a+b) / trimmed_rdd_cnt
        tmp_mse= tmp_trim_rdd.map(lambda line:(line-tmp_mean)*(line-tmp_mean)).reduce(lambda a,b:a+b)/trimmed_rdd_cnt
    
    return (trimmed_rdd_cnt, tmp_mean, tmp_mse, trimmed_rdd)

def split_onOneFeature_exact_trimmedMSE(node_data, node_data_cnt, TMSE_node_ini, feature_valueList, feature_id):
    
    left_count=0
    right_count= node_data_cnt
    bestSplitMetric=  TMSE_node_ini 
    
    #------check the number of feature values------------------
    if len(feature_valueList) <= 1:
        return [-1,-1,-1,-1]
    #-----------------------

    #this is a bit-record variable
    leftSplit_valueSet =0 
    leftSplit_count=0 
    rightSplit_count=0
    
    bestLeft=(-1,-1)
    bestRight=(-1,-1)
    
    #sort feature values according to median 
    sorted_value_map=[]
    
    for i in feature_valueList:
        currentVal = i
        #          adapt to different loss function 
#-----------------------------------        
        tmp_rdd=node_data.filter(lambda l:l[1][feature_id+1] == i ).map(lambda l:l[1][0]).cache()  
        tmp_rdd_count = tmp_rdd.count()
        
        if tmp_rdd_count==0:
            continue
        else:
            tmp_mean = tmp_rdd.reduce(lambda a,b:a+b)*1.0 / tmp_rdd_count
# ------------------------------------
        sorted_value_map.append( (tmp_mean,i) )
        
        
    sorted_value_map.sort()
    len_values= len(sorted_value_map)

    # scan the split positions    
    left_value_set=[]
    right_value_set= copy.deepcopy(feature_valueList)
    
    #--------- initialize the value set ----------------------
    
    # check the number of feature values
    if len_values <= 1:
        return [-1,-1,-1,-1]
    
    current_feature_value = sorted_value_map[0][1] 

    left_value_set.append(current_feature_value)
    right_value_set.remove(current_feature_value)
        
    #          adapt to different loss function     
    left = trimmed_MSE( node_data, feature_id, left_value_set )
    right= trimmed_MSE( node_data, feature_id, right_value_set )
        
    leftSplit_count = left[0]
    leftMedian= left[1]
    leftMetric= left[2]
        
    rightSplit_count= right[0]
    rightMedian= right[1]
    rightMetric= right[2]  
        
    current_splitMetric=1.0*leftSplit_count/node_data_cnt*leftMetric + \
                               1.0*(node_data_cnt - leftSplit_count)/node_data_cnt*rightMetric
        
    bestSplitMetric=current_splitMetric     
    bestLeft=left
    bestRight=right
    #bit variable to record which value is chosen for the left child
    leftSplit_valueSet=leftSplit_valueSet+ (1<<current_feature_value)
    
    #--------------------------------------------------------------
    
    for k in range(1,len_values-1):
        
        current_feature_value = sorted_value_map[k][1] 
        
        left_value_set.append(current_feature_value)
        right_value_set.remove(current_feature_value)
        
#          adapt to different loss function 
        left = trimmed_MSE( node_data, feature_id, left_value_set )
        right= trimmed_MSE( node_data, feature_id, right_value_set )
        
        leftSplit_count = left[0]
        leftMedian= left[1]
        leftMetric= left[2]
        
        rightSplit_count= right[0]
        rightMedian= right[1]
        rightMetric= right[2]  
        
        current_splitMetric=1.0*leftSplit_count/node_data_cnt*leftMetric + \
                               1.0*(node_data_cnt - leftSplit_count)/node_data_cnt*rightMetric
        
        if current_splitMetric < bestSplitMetric:
            bestSplitMetric=current_splitMetric     
            bestLeft=left
            bestRight=right
            #bit variable to record which value is chosen for the left child
            leftSplit_valueSet=leftSplit_valueSet+ (1<<current_feature_value)

    return [ leftSplit_valueSet, bestLeft[1], bestRight[1], bestSplitMetric  ] 
#          value_set, left median, right median, weighted mad



# ------------------------------------------------------------------------------------------------------------

def median_MSE( dataToSplit_rdd, feature_id, value_set ):
    
    tmp_rdd=dataToSplit_rdd.filter(lambda l:l[1][feature_id+1] in value_set).map(lambda l:l[1][0]).cache()
    tmp_rdd_count = tmp_rdd.count()
    
    if tmp_rdd_count==0:
        return (10000, 100000, 10000000)
    
    tmp_mean = tmp_rdd.reduce( lambda a,b: a+b)*1.0/tmp_rdd_count
    tmpMSE= tmp_rdd.map( lambda a: (a-tmp_mean)*(a-tmp_mean) ).reduce( lambda a,b: a+b)*1.0/tmp_rdd_count
    
    return (tmp_rdd_count, tmp_mean, tmpMSE)

def median_MSE_node( dataToSplit_rdd):
    
    tmp_rdd=dataToSplit_rdd.map(lambda l:l[1][0]).cache()
    tmp_rdd_count = dataToSplit_rdd.count()

    if tmp_rdd_count==0:
        return (10000, 100000, 10000000)
    
    tmp_mean = tmp_rdd.reduce( lambda a,b: a+b)*1.0/tmp_rdd_count
    tmpMSE= tmp_rdd.map( lambda a: (a-tmp_mean)*(a-tmp_mean) ).reduce( lambda a,b: a+b)*1.0/tmp_rdd_count
    
    return (tmp_rdd_count, tmp_mean, tmpMSE)

def split_onOneFeature_exact_MSE(node_data, node_data_cnt, MSE_node_ini, feature_valueList, feature_id):
    
    left_count=0
    right_count= node_data_cnt
    bestSplitMetric=  MSE_node_ini 
    
    #------check the number of feature values------------------
    if len(feature_valueList) <= 1:
        return [-1,-1,-1,-1]
    #-----------------------

    #this is a bit-record variable
    leftSplit_valueSet =0 
    leftSplit_count=0  
    rightSplit_count=0
    
    bestLeft=(-1,-1)
    bestRight=(-1,-1)
    
    #sort feature values according to mean 
    sorted_value_map=[]
    
    for i in feature_valueList:
        currentVal = i
#-----------------------------------        
        tmp_rdd=node_data.filter(lambda l:l[1][feature_id+1] == i ).map(lambda l:l[1][0]).cache()  
        tmp_rdd_count = tmp_rdd.count()
    
        if tmp_rdd_count==0:
            continue
        else:
            tmp_mean = tmp_rdd.reduce( lambda a,b: a+b)*1.0/tmp_rdd_count
       
        tmp_mse=(tmp_rdd_count, tmp_mean, 0)
# ------------------------------------
        sorted_value_map.append( (tmp_mse[1],i) )
        
    sorted_value_map.sort()
    len_values= len(sorted_value_map)

    # scan the split positions    
    left_value_set=[]
    right_value_set= copy.deepcopy(feature_valueList)
    
    #--------- initialize the value set ----------------------
    
    # check the number of feature values
    if len_values <= 1:
        return [-1,-1,-1,-1]
    
    current_feature_value = sorted_value_map[0][1] 

    left_value_set.append(current_feature_value)
    right_value_set.remove(current_feature_value)
        
    left = median_MSE( node_data, feature_id, left_value_set )
    right= median_MSE( node_data, feature_id, right_value_set )
        
    leftSplit_count = left[0]
    leftMedian= left[1]
    leftMetric= left[2]
        
    rightSplit_count= right[0]
    rightMedian= right[1]
    rightMetric= right[2]  
        
    #debug
    if leftSplit_count+rightSplit_count!=node_data_cnt:
        print 'problem in left and right'
        
    current_splitMetric=1.0*leftSplit_count/node_data_cnt*leftMetric + \
                               1.0*(node_data_cnt - leftSplit_count)/node_data_cnt*rightMetric
        
    bestSplitMetric=current_splitMetric     
    bestLeft=left
    bestRight=right
    #bit variable to record which value is chosen for the left child
    leftSplit_valueSet=leftSplit_valueSet+ (1<<current_feature_value)
    
    #--------------------------------------------------------------
    
    for k in range(1,len_values-1):
        
        current_feature_value = sorted_value_map[k][1] 
        
        left_value_set.append(current_feature_value)
        right_value_set.remove(current_feature_value)
        
        left = median_MSE( node_data, feature_id, left_value_set )
        right= median_MSE( node_data, feature_id, right_value_set )
        
        leftSplit_count = left[0]
        leftMedian= left[1]
        leftMetric= left[2]
        
        rightSplit_count= right[0]
        rightMedian= right[1]
        rightMetric= right[2]  
        
        #debug
        if leftSplit_count+rightSplit_count!=node_data_cnt:
            print 'problem in left and right'
        
        
        current_splitMetric=1.0*leftSplit_count/node_data_cnt*leftMetric + \
                               1.0*(node_data_cnt - leftSplit_count)/node_data_cnt*rightMetric
        
        if current_splitMetric < bestSplitMetric:
            bestSplitMetric=current_splitMetric     
            bestLeft=left
            bestRight=right
            #bit variable to record which value is chosen for the left child
            leftSplit_valueSet=leftSplit_valueSet+ (1<<current_feature_value)

    return [ leftSplit_valueSet, bestLeft[1], bestRight[1], bestSplitMetric  ] 
#          value_set, left median, right median, weighted mad

# -----------------------------------------------------------------------------------------------------------

#calculate median and MAD for a set of data in dataToSplit_rdd

def median_MAD_cal( dataToSplit_rdd ):
    
    tmp_rdd=dataToSplit_rdd
    tmp_rdd_count = tmp_rdd.count()
    
    if tmp_rdd_count==0:
        return (10000, 100000, 10000000)
    
#     optimization
    sorted_rdd = tmp_rdd.sortBy().zipWithIndex().map(lambda line: (line[1], line[0]) )
    
    
    
    if(tmp_rdd_count%2 ==0):
        tmphalf = tmp_rdd.top( tmp_rdd_count/2+1  )
        tmphalf.sort()
        tmpcnt= len(tmphalf)
        tmp_median = (tmphalf[0] + tmphalf[1])*1.0/2.0
    else:
        tmphalf = tmp_rdd.top( tmp_rdd_count/2+1  )
        tmphalf.sort()
        tmpcnt= len(tmphalf)
        tmp_median = tmphalf[0]
    
    tmpMAD_rdd = tmp_rdd.map(lambda line: abs(line-tmp_median) ).cache()
    
    if(tmp_rdd_count%2 ==0):
        tmphalf = tmpMAD_rdd.top( tmp_rdd_count/2+1  )
        tmpcnt= len(tmphalf)
        tmp_mad = (tmphalf[tmpcnt-1] + tmphalf[tmpcnt-2])*1.0/2.0
    else:
        tmphalf = tmpMAD_rdd.top( tmp_rdd_count/2+1  )
        tmpcnt= len(tmphalf)
        tmp_mad = tmphalf[tmpcnt-1]
    
    return (tmp_rdd_count, tmp_median, tmp_mad)


def median_MAD( dataToSplit_rdd, feature_id, value_set ):
    
    tmp_rdd=dataToSplit_rdd.filter(lambda l:l[1][feature_id+1] in value_set).map(lambda l:l[1][0]).cache()
    tmp_rdd_count = tmp_rdd.count()
    return  median_MAD_cal( tmp_rdd )

def median_MAD_node( dataToSplit_rdd):
    
    tmp_rdd=dataToSplit_rdd.map(lambda l:l[1][0]).cache()
    tmp_rdd_count = dataToSplit_rdd.count()
    return  median_MAD_cal( tmp_rdd )

def split_onOneFeature_exact_MAD(node_data, node_data_cnt, node_ini, feature_valueList, feature_id):
    
    left_count=0
    right_count= node_data_cnt
    bestSplitMetric=  node_ini 
    
    #------check the number of feature values------------------
    if len(feature_valueList) <= 1:
        return [-1,-1,-1,-1]
    #-----------------------

    #this is a bit-record variable
    leftSplit_valueSet =0 
    leftSplit_count=0 
    rightSplit_count=0
    
    bestLeft=(-1,-1)
    bestRight=(-1,-1)
    
    #sort feature values according to median 
    sorted_value_map=[]
    
    for i in feature_valueList:
        currentVal = i
#         dataToFeatureValue_rdd=node_data.filter(lambda l:l[1][feature_id+1]==currentVal).map(lambda l:l[1][0]).cache()
#         dataToFeatureValue_rdd_count = dataToFeatureValue_rdd.count()

#-----------------------------------        
        tmp_rdd=node_data.filter(lambda l:l[1][feature_id+1] == i ).map(lambda l:l[1][0]).cache()  
        tmp_rdd_count = tmp_rdd.count()   
        tmp_mad=  median_MAD_cal( tmp_rdd )
# ------------------------------------
        sorted_value_map.append( (tmp_mad[1],i) )
        
    sorted_value_map.sort()
    len_values= len(sorted_value_map)

    # scan the split positions    
    left_value_set=[]
    right_value_set= copy.deepcopy(feature_valueList)
    
    #--------- initialize the value set ----------------------
    
    # check the number of feature values
    if len_values <= 1:
        return [-1,-1,-1,-1]
    
    current_feature_value = sorted_value_map[0][1] 

    left_value_set.append(current_feature_value)
    right_value_set.remove(current_feature_value)
        
    left = median_MAD( node_data, feature_id, left_value_set )
    right= median_MAD( node_data, feature_id, right_value_set )
        
    leftSplit_count = left[0]
    leftMedian= left[1]
    leftMetric= left[2]
        
    rightSplit_count= right[0]
    rightMedian= right[1]
    rightMetric= right[2]  
        
    current_splitMetric=1.0*leftSplit_count/node_data_cnt*leftMetric + \
                               1.0*(node_data_cnt - leftSplit_count)/node_data_cnt*rightMetric
        
    bestSplitMetric=current_splitMetric     
    bestLeft=left
    bestRight=right
    #bit variable to record which value is chosen for the left child
    leftSplit_valueSet=leftSplit_valueSet+ (1<<current_feature_value)
    
    #--------------------------------------------------------------
    
    for k in range(1,len_values-1):
        
        current_feature_value = sorted_value_map[k][1] 
        
        left_value_set.append(current_feature_value)
        right_value_set.remove(current_feature_value)
        
        left = median_MAD( node_data, feature_id, left_value_set )
        right= median_MAD( node_data, feature_id, right_value_set )
        
        leftSplit_count = left[0]
        leftMedian= left[1]
        leftMetric= left[2]
        
        rightSplit_count= right[0]
        rightMedian= right[1]
        rightMetric= right[2]  
        
        current_splitMetric=1.0*leftSplit_count/node_data_cnt*leftMetric + \
                               1.0*(node_data_cnt - leftSplit_count)/node_data_cnt*rightMetric
        
        if current_splitMetric < bestSplitMetric:
            bestSplitMetric=current_splitMetric     
            bestLeft=left
            bestRight=right
            #bit variable to record which value is chosen for the left child
            leftSplit_valueSet=leftSplit_valueSet+ (1<<current_feature_value)

    return [ leftSplit_valueSet, bestLeft[1], bestRight[1], bestSplitMetric  ] 
#          value_set, left median, right median, weighted mad


# -----------------------------------------------------------------------------------------------------------

#calculate median and MAD for a set of data in dataToSplit_rdd
def median_LAD( dataToSplit_rdd, feature_id, value_set ):
    
    tmp_rdd=dataToSplit_rdd.filter(lambda l:l[1][feature_id+1] in value_set).map(lambda l:l[1][0]).cache()
    tmp_rdd_count = tmp_rdd.count()
    
    if tmp_rdd_count==0:
        return (10000, 100000, 10000000)
    
#----------median computation optimization---------------------------------
    sorted_rdd = tmp_rdd.sortBy(lambda line: line).zipWithIndex().map(lambda line: (line[1], line[0]) )
    if(tmp_rdd_count%2 ==0):
        tmp1=sorted_rdd.lookup( tmp_rdd_count/2-1  )[0]
        tmp2=sorted_rdd.lookup( tmp_rdd_count/2  )[0]
        tmp_median = (tmp1 + tmp1)*1.0/2.0
    else:
        tmp_median = sorted_rdd.lookup( tmp_rdd_count/2  )[0]
#-------------------------------------------------------------
    
#     if(tmp_rdd_count%2 ==0):
#         tmphalf = tmp_rdd.top( tmp_rdd_count/2+1  )
#         tmphalf.sort()
#         tmpcnt= len(tmphalf)
#         tmp_median = (tmphalf[0] + tmphalf[1])*1.0/2.0
#     else:
#         tmphalf = tmp_rdd.top( tmp_rdd_count/2+1  )
#         tmphalf.sort()
#         tmpcnt= len(tmphalf)
#         tmp_median = tmphalf[0]
    
    tmpMAD_rdd = tmp_rdd.map(lambda line: abs(line-tmp_median) ).cache()
    tmpMAD = tmpMAD_rdd.reduce( lambda a,b: a+b)*1.0 / tmp_rdd_count
    
    return (tmp_rdd_count, tmp_median, tmpMAD)

def median_LAD_node( dataToSplit_rdd):
    
    tmp_rdd=dataToSplit_rdd.map(lambda l:l[1][0]).cache()
    tmp_rdd_count = dataToSplit_rdd.count()
 
    if tmp_rdd_count==0:
        return (10000, 100000, 10000000)
    
    #----------median computation optimization---------------------------------
    sorted_rdd = tmp_rdd.sortBy(lambda line: line).zipWithIndex().map(lambda line: (line[1], line[0]) )
    if(tmp_rdd_count%2 ==0):
        tmp1=sorted_rdd.lookup( tmp_rdd_count/2-1  )[0]
        tmp2=sorted_rdd.lookup( tmp_rdd_count/2  )[0]
        tmp_median = (tmp1 + tmp1)*1.0/2.0
    else:
        tmp_median = sorted_rdd.lookup( tmp_rdd_count/2  )[0]
    #-------------------------------------------------------------
    
#     if(tmp_rdd_count%2 ==0):
#         tmphalf = tmp_rdd.top( tmp_rdd_count/2+1  )
#         tmphalf.sort()
#         tmpcnt= len(tmphalf)
#         tmp_median = (tmphalf[0] + tmphalf[1])*1.0/2.0
#     else:
#         tmphalf = tmp_rdd.top( tmp_rdd_count/2+1  )
#         tmphalf.sort()
#         tmpcnt= len(tmphalf)
#         tmp_median = tmphalf[0]
    
    tmpMAD_rdd = tmp_rdd.map(lambda line: abs(line-tmp_median) ).cache()
    tmpMAD = tmpMAD_rdd.reduce( lambda a,b: a+b)*1.0 / tmp_rdd_count
    
    return (tmp_rdd_count, tmp_median, tmpMAD)

def split_onOneFeature_exact_LAD(node_data, node_data_cnt, LAD_node_ini, feature_valueList, feature_id):
    
    left_count=0
    right_count= node_data_cnt
    bestSplitMetric=  LAD_node_ini 
    
    #------check the number of feature values------------------
    if len(feature_valueList) <= 1:
        return [-1,-1,-1,-1]
    #-----------------------

    #this is a bit-record variable
    leftSplit_valueSet =0 
    leftSplit_count=0 
    rightSplit_count=0
    
    bestLeft=(-1,-1)
    bestRight=(-1,-1)
    
    #sort feature values according to median 
    sorted_value_map=[]
    tmpvalues=[]
    
    for i in feature_valueList:
#         dataToFeatureValue_rdd=node_data.filter(lambda l:l[1][feature_id+1]==currentVal).map(lambda l:l[1][0]).cache()
#         dataToFeatureValue_rdd_count = dataToFeatureValue_rdd.count()

#-----------------------------------        
        tmp_rdd=node_data.filter(lambda l:l[1][feature_id+1] == i ).map(lambda l:l[1][0]).cache()  
        tmp_rdd_count = tmp_rdd.count()
    
        if tmp_rdd_count==0:
            continue
        else:
            sorted_rdd = tmp_rdd.sortBy(lambda line: line).zipWithIndex().map(lambda line: (line[1], line[0]) )
            if(tmp_rdd_count%2 ==0):
                tmp1=sorted_rdd.lookup( tmp_rdd_count/2-1  )[0]
                tmp2=sorted_rdd.lookup( tmp_rdd_count/2  )[0]
                tmp_median = (tmp1 + tmp1)*1.0/2.0
            else:
                tmp_median = sorted_rdd.lookup( tmp_rdd_count/2  )[0]
    
#             tmpMAD_rdd = tmp_rdd.map(lambda line: abs(line-tmp_median) ).cache()
#             tmpMAD = tmpMAD_rdd.reduce( lambda a,b: a+b)
       
        tmp_mad=(tmp_rdd_count, tmp_median, 0)
        
#         del tmpvalues[:]
#         tmpvalues.append(i)
#         tmp_mad = median_LAD( node_data, i, tmpvalues )
        
# ------------------------------------
        sorted_value_map.append( (tmp_mad[1],i) )
        
    sorted_value_map.sort()
    len_values= len(sorted_value_map)

    # scan the split positions    
    left_value_set=[]
    right_value_set= copy.deepcopy(feature_valueList)
    
    #--------- initialize the value set ----------------------
    
    # check the number of feature values
    if len_values <= 1:
        return [-1,-1,-1,-1]
    
    current_feature_value = sorted_value_map[0][1] 

    left_value_set.append(current_feature_value)
    right_value_set.remove(current_feature_value)
        
    left = median_LAD( node_data, feature_id, left_value_set )
    right= median_LAD( node_data, feature_id, right_value_set )
        
    leftSplit_count = left[0]
    leftMedian= left[1]
    leftMetric= left[2]
        
    rightSplit_count= right[0]
    rightMedian= right[1]
    rightMetric= right[2]  
        
    current_splitMetric=1.0*leftSplit_count/node_data_cnt*leftMetric + \
                               1.0*(node_data_cnt - leftSplit_count)/node_data_cnt*rightMetric
        
    bestSplitMetric=current_splitMetric     
    bestLeft=left
    bestRight=right
    #bit variable to record which value is chosen for the left child
    leftSplit_valueSet=leftSplit_valueSet+ (1<<current_feature_value)
    
    #--------------------------------------------------------------
    
    for k in range(1,len_values-1):
        
        current_feature_value = sorted_value_map[k][1] 
        
        left_value_set.append(current_feature_value)
        right_value_set.remove(current_feature_value)
        
        left = median_LAD( node_data, feature_id, left_value_set )
        right= median_LAD( node_data, feature_id, right_value_set )
        
        leftSplit_count = left[0]
        leftMedian= left[1]
        leftMetric= left[2]
        
        rightSplit_count= right[0]
        rightMedian= right[1]
        rightMetric= right[2]  
        
        current_splitMetric=1.0*leftSplit_count/node_data_cnt*leftMetric + \
                               1.0*(node_data_cnt - leftSplit_count)/node_data_cnt*rightMetric
        
        if current_splitMetric < bestSplitMetric:
            bestSplitMetric=current_splitMetric     
            bestLeft=left
            bestRight=right
            #bit variable to record which value is chosen for the left child
            leftSplit_valueSet=leftSplit_valueSet+ (1<<current_feature_value)

    return [ leftSplit_valueSet, bestLeft[1], bestRight[1], bestSplitMetric  ] 
#          value_set, left median, right median, weighted mad

def find_bestSplit_exact( dataToNode, current_NumNodes, nodes_tree, nodes_tree_test, feature_valueList, loss_func):
    
    tmpnode_cnt=[]
    presplit=[]
        
#------------- grow the unbalanced tree------------------------    
    cur_node_num= len(nodes_tree)
# -------------------------------------------------------------
        
    for i in range(0, current_NumNodes):
    
    #------------- grow the unbalanced tree ---------------------------------- 
        if cur_node_num >= 3:
            parent_node = int(math.ceil(  (i + cur_node_num -2.0)*1.0/ 2 ))
            parent_node_esti =nodes_tree_test[2* parent_node +1 ] 
            tmp_split_feature= nodes_tree[parent_node][0]
            
            if tmp_split_feature == -1:
                nodes_tree.append(  (-1,  -1  )  )
                nodes_tree_test.append( parent_node_esti  )
                nodes_tree_test.append( parent_node_esti )
                continue
    #-------------------------------------------------------------------------
        # prepare data for this node
        current_node=i
        current_node_data = dataToNode.filter( lambda line: line[0] == current_node ).cache()
        current_node_data_cnt = current_node_data.count()
        
        # split initialization  
        if loss_func == 'tmse':
            
            if abs(trimm_ratio-0.0)<1e-9:
                
                trimmed_rdd= current_node_data
                trimmed_rdd_cnt = trimmed_rdd.count()
                    
                tmp_rdd=current_node_data.map(lambda l:l[1][0]).cache()
            
                tmp_mean = tmp_rdd.reduce(lambda a,b:a+b) / trimmed_rdd_cnt
                tmp_mse= tmp_rdd.map(lambda line:(line-tmp_mean)*(line-tmp_mean)).reduce(lambda a,b:a+b)/trimmed_rdd_cnt
                
                
                tmp_metric=(trimmed_rdd_cnt, tmp_mean, tmp_mse, trimmed_rdd )
                
            else:
                tmp_metric =  trimmed_MSE_node( current_node_data, trimm_ratio )
                
                
            best_splitMetric_sofar= tmp_metric[2]
                
#                  (trimmed_rdd_cnt, tmp_mean, tmp_mse, trimmed_rdd)
                
#                 tmp_rdd=current_node_data.map(lambda l:l[1][0]).cache()
#                 tmp_rdd_count = tmp_rdd.count()
    
#                 upper_bound = upper_tail_boundary( trimm_ratio, tmp_rdd, tmp_rdd_count)
#         #     lower_bound = lower_tail_boundary( trim_percentage, dat_rdd, dat_rdd_cnt)
#                 tmp_trim_rdd= tmp_rdd.filter( lambda line: line < upper_bound ).cache()
 
#                 trimmed_rdd= current_node_data.filter( lambda line: line[1][0] < upper_bound ).cache()
#         #     trimmed_rdd= trimmed_rdd.filter(lambda line: line > lower_bound).cache()
#                 trimmed_rdd_cnt = trimmed_rdd.count()
            
#                 if trimmed_rdd_cnt ==0:
#                     print 'trimmed zeor happens!'
#                     tmp_mean=0
#                     tmp_mse=0
#                 else:
#                     tmp_mean = tmp_trim_rdd.reduce(lambda a,b:a+b) / trimmed_rdd_cnt
#                     tmp_mse= tmp_trim_rdd.map(lambda line:(line-tmp_mean)*(line-tmp_mean)).reduce(lambda a,b:a+b)/trimmed_rdd_cnt
                
#             tmp_metric=(trimmed_rdd_cnt,tmp_mean,tmp_mse)
            
            
    
        elif loss_func == 'lad':
                tmp_metric = median_LAD_node( current_node_data)
                best_splitMetric_sofar= tmp_metric[2]
        elif loss_func == 'mse':
                tmp_metric = median_MSE_node( current_node_data)
                best_splitMetric_sofar= tmp_metric[2]
        elif loss_func == 'mad':
                tmp_metric = median_MAD_node( current_node_data)
                best_splitMetric_sofar= tmp_metric[2]
        
        
        best_split=(-1,-1,-1,-1)
        best_split_feature=-1
        best_split_featureValueSet=-1
                
        #debug
        tmpnode_cnt.append( current_node_data.count()  )
        presplit.append( best_splitMetric_sofar/100000 )
        
        for j in range(0,numFeatures):    
                                    
            if loss_func == 'tmse':
                
                current_node_data= tmp_metric[3]
                current_node_data_cnt= tmp_metric[0]
                
                cur_split=split_onOneFeature_exact_trimmedMSE(current_node_data,current_node_data_cnt,\
                                           best_splitMetric_sofar,feature_valueList[j][1],j)
            elif loss_func == 'lad':
                 cur_split=split_onOneFeature_exact_LAD(current_node_data,current_node_data_cnt,\
                                           best_splitMetric_sofar,feature_valueList[j][1],j)
            elif loss_func == 'mad':
                 cur_split=split_onOneFeature_exact_MAD(current_node_data,current_node_data_cnt,\
                                           best_splitMetric_sofar,feature_valueList[j][1],j)
            elif loss_func == 'mse':
                 cur_split=split_onOneFeature_exact_MSE(current_node_data,current_node_data_cnt,\
                                           best_splitMetric_sofar,feature_valueList[j][1],j)
                
            if  cur_split[0]!=-1  and cur_split[3] < best_splitMetric_sofar:
                best_split=cur_split
                best_splitMetric_sofar=cur_split[3]
                best_split_feature = j
                best_split_featureValueSet= cur_split[0]                  
                
#------------- grow the unbalanced tree ---------------------------------- 
        if best_split_feature == -1:
            nodes_tree.append(  (-1,  -1  )  )
            nodes_tree_test.append( tmp_metric[1]  )
            nodes_tree_test.append( tmp_metric[1] )
            continue
#-------------------------------------------------------------------------

        # tree for spliting nodes        
        nodes_tree.append(  (best_split_feature,  best_split_featureValueSet  )  )
        
        # tree for predicting   
        nodes_tree_test.append(best_split[1])
        nodes_tree_test.append(best_split[2])
        
#-----debug---------
#     print tmpnode_cnt
#     print presplit
    
#     return nodes_layer

In [6]:
# training and testing process: data preparation

# with outliers
# tmpdta = dta.takeSample(False, 1000000, 12243)
# dta_train_all = sc.parallelize(tmpdta).cache().sample(False, .1, 12345)
# dta_test_all = sc.parallelize(tmpdta).cache().sample(False, .1, 43243)


dta_train_all = dta.cache().sample(False, .3, 12345)
dta_test_all = dta.cache().sample(False, .1, 43243)

print dta_train_all.count(), dta_test_all.count() 
# configurate extraction
# print 'number of feature-value combinations:',len(dta_train_all.map(lambda line:(line[1],line[2],line[3])).distinct().collect())


994511 332190


In [7]:
# training and testing process: clearned or noisy data ?

# 98974
# 99987

dta_train = dta_train_all
# .filter(lambda line: line[0]<1000 )
dta_test = dta_test_all.filter(lambda line: line[0]<1000 )

print dta_train.count()
print dta_test.count()
# print 'number of feature-value combinations:',len(dta_train.map(lambda line:(line[1],line[2],line[3])).distinct().collect())

994511
312746


In [ ]:
# training and testing process: main process

#parameters
maxdepth=8
numFeatures=7
trimm_ratio = 0.00
# tmse (trimmed mse), lad, ma d, mse
loss_func= 'lad'

def tree_test_mapFunc_median(line):
    tmpnode = search_nodeToData(line[1:numFeatures+1], node_split)
    return (leaf_nodes[ tmpnode ]  - line[0])*(leaf_nodes[ tmpnode ]  - line[0])

def tree_test_mapFunc(line):
    tmpnode = search_nodeToData(line[1:numFeatures+1], node_split)
    return (leaf_nodes[ tmpnode ]  - line[0])*(leaf_nodes[ tmpnode ]  - line[0])
    
def tree_test( testData_rdd ):
    err_rdd=testData_rdd.map( lambda line: tree_test_mapFunc(line))
    err_sum = err_rdd.reduce( lambda a,b: a+b)
    return err_sum / testData_rdd.count()

# variables for the tree structure 
node_split=[]
node_test=[]

test_err = []
train_err = []
run_time=[]

tree_history_split=[]
tree_history_esti=[]
tree_history_leaf=[]
tree_history_runtime=[]


#ini parameter on the cluster
currentNode_split_fromMaster = sc.broadcast([])


#prepare feature value set
feature_valueSet = data_featureValues_collect( dta_train )
feature_valueList=[]
for i in range(0, len(feature_valueSet)):
    feature_valueList.append( ( i, list(feature_valueSet[i][1]) ) )
    
#     tune the starting depth
for i in range(maxdepth,maxdepth+1):
    
    start = time.time() 
    
    del node_split[:]
    del node_test[:]
    currentNode_split_fromMaster = sc.broadcast([])
    
    for cur_depth in range(0,i):
        
        current_nodeNum= int( math.pow(2,  cur_depth)) 
        dataToNode = dataToNode_assignment( dta_train )
        
        #debug
        print  dataToNode.map(lambda line:line[0]).distinct().count() , current_nodeNum
        
        find_bestSplit_exact(dataToNode,current_nodeNum, node_split, node_test, feature_valueList,loss_func)   
#         ( dataToNode, current_NumNodes, nodes_tree, nodes_tree_test, feature_valueSet):
        #for whole tree
        currentNode_split_fromMaster = sc.broadcast(node_split)
    
    end = time.time()
    leaf_nodes = node_test[ len(node_test)-  (int)( math.pow(2,i)): len(node_test)  ]
    
    tmp_test_err= 0 
#     tree_test( dta_test )
    tmp_train_err= 0
#     tree_test( dta_train )
#     test_err.append( tmp_test_err)
#     train_err.append( tmp_train_err )

    elapsed = end-start
    run_time.append(elapsed)

    tree_history_split.append(copy.deepcopy(node_split) )
    tree_history_esti.append( copy.deepcopy(node_test)  )
    tree_history_leaf.append( copy.deepcopy(leaf_nodes)  )
    tree_history_runtime.append( copy.deepcopy(run_time)  )
        
    print "error at tree height", i,":",  tmp_test_err, tmp_train_err
    print "running time at tree height", i,":",  elapsed
    print "number of leaf nodes at tree height", i,":",  len(leaf_nodes)
    

In [238]:
tmp_split=tree_history_split
tmp_esti= tree_history_esti
tmp_leaf= tree_history_leaf

In [22]:
print dta_test.first()
err_rdd=dta_test.map(lambda line:search_nodeToData(line[1:numFeatures+1], node_split) ) 
print len(node_split)
print len(node_test)
print len(leaf_nodes)

# print node_split

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: java.lang.IllegalStateException: SparkContext has been shutdown
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:1726)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:1752)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:1767)
	at org.apache.spark.api.python.PythonRDD$.runJob(PythonRDD.scala:366)
	at org.apache.spark.api.python.PythonRDD.runJob(PythonRDD.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:483)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:231)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:379)
	at py4j.Gateway.invoke(Gateway.java:259)
	at py4j.commands.AbstractCommand.inv

In [215]:
err_rdd=dta_test.map(lambda line:(line[0],leaf_nodes[search_nodeToData(line[1:numFeatures+1], node_split) ], search_nodeToData(line[1:numFeatures+1], node_split) ) )

print err_rdd.map( lambda line: (line[0]-line[1])*(line[0]-line[1])  ).reduce(lambda a, b: a+b)/err_rdd.count()
# err_rdd.collect()

0.995640799666


In [46]:
# training and testing process: extract all the feature-value combinations 
# in the training data set

total_featureVal_set=[]
for i in range(0, numFeatures):
    featureValues = dta_train.map(lambda line: line[1+i]).distinct().collect()
    total_featureVal_set.append( featureValues)

print 'done'


done


In [13]:
#---------------- LAD -------------------  data backup
lad_tree_history_split = copy.deepcopy( tree_history_split)
lad_tree_history_esti =  copy.deepcopy( tree_history_esti)
lad_tree_history_leaf =  copy.deepcopy( tree_history_leaf)
lad_tree_history_runtime =  copy.deepcopy( tree_history_runtime)

print len(lad_tree_history_split)
print len(lad_tree_history_split[0])
print len(lad_tree_history_esti)
print len(lad_tree_history_esti[0])

print len(lad_tree_history_leaf)
print len(lad_tree_history_leaf[0])

print lad_tree_history_runtime

1
15
1
30
1
16
[[4014.9761939048767]]


In [18]:
lad_tree_history_esti

[[312.0,
  426.0,
  294.0,
  333.0,
  389.0,
  505.0,
  263.0,
  304.0,
  323.0,
  345.0,
  376.0,
  420.0,
  500.0,
  1150.0,
  244.0,
  295.0,
  266.0,
  308.0,
  316.0,
  346.0,
  344.0,
  505.0,
  359.0,
  393.0,
  405.0,
  455.0,
  404.0,
  508.0,
  504.0,
  1150.0]]

In [ ]:
# LAD result statistic: node infor.

# lad_tree_history_split = copy.deepcopy( tree_history_split)
# lad_tree_history_esti =  copy.deepcopy( tree_history_esti)

print 'test error:',test_err
print 'training error:',train_err
print len(lad_tree_history_split)

recog_conf=bfs_tree(lad_tree_history_split[ 0 ], total_featureVal_set, maxdepth-1,  lad_tree_history_esti[0], dta_train )  

print len(recog_conf[0])
print recog_conf[1]
print recog_conf[2]

In [16]:
# LAD result statistic : test error

pre=0
tree_esti = lad_tree_history_esti[0]
tree_split =  lad_tree_history_split[ 0 ]

for i in range(1,maxdepth+1):
            
    end_bound= pre+ (int)( math.pow(2,i)) 
    test_leaf_nodes = tree_esti[ pre: end_bound  ]
    pre= end_bound        
    test_tree_split= tree_split[ 0:(int)( math.pow(2,i)-1)  ]
    
#     print len(test_tree_split)
#     print len(test_leaf_nodes)

    test_error_tree( i, dta_test  )

test error at depth 1 : 19766.1169192
test error at depth 2 : 18967.6044969
test error at depth 3 : 48333.2289398
test error at depth 4 : 19889.3832151


In [202]:
#--------------- MAD --------------- result backup

mad_tree_history_split = copy.deepcopy( tree_history_split)
mad_tree_history_esti =  copy.deepcopy( tree_history_esti)
mad_tree_history_leaf =  copy.deepcopy( tree_history_leaf)
mad_tree_history_runtime =  copy.deepcopy( tree_history_runtime)


print len(mad_tree_history_split)
print len(mad_tree_history_split[0])
print len(mad_tree_history_esti)
print len(mad_tree_history_esti[0])

print len(mad_tree_history_leaf)
print len(mad_tree_history_leaf[0])

1 1


In [ ]:
# MAD result statistic
# mad_tree_history_split = copy.deepcopy( tree_history_split)
# mad_tree_history_esti =  copy.deepcopy( tree_history_esti)

# for i in range(1, maxdepth):
#     print '-------tree with depth: ',i,'-----------'
#     bfs_tree(mse_tree_history_esti[i], total_featureVal_set, i)

print 'test error:',test_err
print 'training error:',train_err
print len(mad_tree_history_split)

bfs_tree(mad_tree_history_split[ maxdepth-1 ], total_featureVal_set, maxdepth-1)    

In [ ]:
# MAD result statistic : test error

pre=0
tree_esti = mad_tree_history_esti[0]
tree_split =  mad_tree_history_split[ 0 ]

for i in range(1,maxdepth+1):
            
    end_bound= pre+ (int)( math.pow(2,i)) 
    test_leaf_nodes = tree_esti[ pre: end_bound  ]
    pre= end_bound        
    test_tree_split= tree_split[ 0:(int)( math.pow(2,i)-1)  ]
    
#     print len(test_tree_split)
#     print len(test_leaf_nodes)

    test_error_tree( i, dta_test  )

In [11]:
#---------------MSE-------------- result backup

mse_tree_history_split = copy.deepcopy( tree_history_split)
mse_tree_history_esti =  copy.deepcopy( tree_history_esti)
mse_tree_history_leaf =  copy.deepcopy( tree_history_leaf)
mse_tree_history_runtime =  copy.deepcopy( tree_history_runtime)


print len(mse_tree_history_split)
print len(mse_tree_history_split[0])
print len(mse_tree_history_esti)
print len(mse_tree_history_esti[0])

print len(mse_tree_history_leaf)
print len(mse_tree_history_leaf[0])

1
1023
1
2046
1
1024


In [14]:
# MSE result statistic

# mse_tree_history_split = copy.deepcopy( tree_history_split)
# mse_tree_history_esti =  copy.deepcopy( tree_history_esti)

# for i in range(1, maxdepth):
#     print '-------tree with depth: ',i,'-----------'
#     bfs_tree(mse_tree_history_esti[i], total_featureVal_set, i)

print 'test error:',test_err
print 'training error:',train_err
print len(mse_tree_history_split)

recog_conf = bfs_tree(mse_tree_history_split[ 0 ], total_featureVal_set, maxdepth-1,  mse_tree_history_esti[0], dta_train )

print len(recog_conf[0])
print recog_conf[1]
print recog_conf[2]


test error: []
training error: []
1
non_leaf_node count at previous depth 0
depth 0 : 1
[[3, 6, 4, 2, 5], [12, 15, 13, 11, 14], [105, 102, 103, 104, 101]] non-leaf node
non_leaf_node count at previous depth 1
depth 1 : 2
[[2], [12, 15, 13, 11, 14], [105, 102, 103, 104, 101]] non-leaf node
[[3, 4, 5, 6], [12, 15, 13, 11, 14], [105, 102, 103, 104, 101]] non-leaf node
non_leaf_node count at previous depth 2
depth 2 : 4
[[2], [12, 15, 13, 11, 14], [104, 105]] non-leaf node
[[2], [12, 15, 13, 11, 14], [101, 102, 103]] non-leaf node
[[3, 4, 5, 6], [15], [105, 102, 103, 104, 101]] non-leaf node
[[3, 4, 5, 6], [11, 12, 13, 14], [105, 102, 103, 104, 101]] non-leaf node
non_leaf_node count at previous depth 4
depth 3 : 8
[[2], [11, 13, 14], [104, 105]] non-leaf node
[[2], [12, 15], [104, 105]] non-leaf node
[[2], [12, 15, 13, 11, 14], [102, 103]] non-leaf node
[[2], [12, 15, 13, 11, 14], [101]] non-leaf node
[[6], [15], [105, 102, 103, 104, 101]] non-leaf node
[[3, 4, 5], [15], [105, 102, 103, 1

In [15]:
# MSE result statistic : test error

pre=0
tree_esti = mse_tree_history_esti[0]
tree_split =  mse_tree_history_split[ 0 ]
test_error_depth = []

for i in range(1,maxdepth+1):
            
    end_bound= pre+ (int)( math.pow(2,i)) 
    test_leaf_nodes = tree_esti[ pre: end_bound  ]
    pre= end_bound        
    test_tree_split= tree_split[ 0:(int)( math.pow(2,i)-1)  ]
#     print len(test_tree_split)
#     print len(test_leaf_nodes)
#     test_error_tree( i, dta_test  )
    
    test_error_depth.append( test_error_tree( i, dta_test  ) )

print test_error_depth

test error at depth 1 : 10951.9519154
test error at depth 2 : 10874.4913409
test error at depth 3 : 10898.5890795
test error at depth 4 : 11074.6896286
test error at depth 5 : 11027.8917796
test error at depth 6 : 11258.8055739
test error at depth 7 : 11308.4776686
test error at depth 8 : 11397.2901348
test error at depth 9 : 11471.4427182
test error at depth 10 : 11477.0047595
[10951.951915391994, 10874.491340893754, 10898.589079538771, 11074.689628642509, 11027.89177964135, 11258.80557388515, 11308.477668631882, 11397.29013480827, 11471.442718201137, 11477.004759472811]


In [51]:
#--------------TMSE-------: result backup

tmse_tree_history_split = copy.deepcopy( tree_history_split)
tmse_tree_history_esti =  copy.deepcopy( tree_history_esti)
tmse_tree_history_leaf =  copy.deepcopy( tree_history_leaf)
tmse_tree_history_runtime =  copy.deepcopy( tree_history_runtime)


print len(tmse_tree_history_split)
print len(tmse_tree_history_split[0])
print len(tmse_tree_history_esti)
print len(tmse_tree_history_esti[0])

print len(tmse_tree_history_leaf)
print len(tmse_tree_history_leaf[0])

1
1023
1
2046
1
1024


In [53]:
# TMSE result statistic

# tmse_tree_history_split = copy.deepcopy( tree_history_split)
# tmse_tree_history_esti =  copy.deepcopy( tree_history_esti)

# for i in range(1, maxdepth):
#     print '-------tree with depth: ',i,'-----------'
#     bfs_tree(mse_tree_history_esti[i], total_featureVal_set, i)

print 'test error:',test_err
print 'training error:',train_err
print len(tmse_tree_history_split)   

recog_conf=bfs_tree(tmse_tree_history_split[ 0 ],total_featureVal_set,maxdepth-1,tmse_tree_history_esti[0],dta_train ) 

print len(recog_conf[0])
print recog_conf[1]
print recog_conf[2]

test error: []
training error: []
1
non_leaf_node count at previous depth 0
depth 0 : 1
[[3, 6, 4, 2, 5], [12, 15, 13, 11, 14], [105, 102, 103, 104, 101]] non-leaf node
non_leaf_node count at previous depth 1
depth 1 : 2
[[3, 6, 4, 2, 5], [15], [105, 102, 103, 104, 101]] non-leaf node
[[3, 6, 4, 2, 5], [11, 12, 13, 14], [105, 102, 103, 104, 101]] non-leaf node
non_leaf_node count at previous depth 2
depth 2 : 4
[[3, 6], [15], [105, 102, 103, 104, 101]] non-leaf node
[[2, 4, 5], [15], [105, 102, 103, 104, 101]] non-leaf node
[[3, 6, 4, 2, 5], [11, 12, 13, 14], [104, 105]] non-leaf node
[[3, 6, 4, 2, 5], [11, 12, 13, 14], [101, 102, 103]] non-leaf node
non_leaf_node count at previous depth 4
depth 3 : 8
[[3, 6], [15], [101, 102, 103, 104]] non-leaf node
[[3, 6], [15], [105]] non-leaf node
[[2, 4, 5], [15], [101, 103, 104, 105]] non-leaf node
[[2, 4, 5], [15], [102]] non-leaf node
[[2], [11, 12, 13, 14], [104, 105]] non-leaf node
[[3, 4, 5, 6], [11, 12, 13, 14], [104, 105]] non-leaf node


In [52]:
# TMSE result statistic : test error

pre=0
tree_esti = tmse_tree_history_esti[0]
tree_split =  tmse_tree_history_split[ 0 ]

test_error_depth=[]

for i in range(1,maxdepth+1):
            
    end_bound= pre+ (int)( math.pow(2,i)) 
    test_leaf_nodes = tree_esti[ pre: end_bound  ]
    pre= end_bound        
    test_tree_split= tree_split[ 0:(int)( math.pow(2,i)-1)  ]
    
#     print len(test_tree_split)
#     print len(test_leaf_nodes)

    test_error_depth.append( test_error_tree( i, dta_test  ) )

print test_error_depth

test error at depth 1 : 722.061178985
test error at depth 2 : 677.890121689
test error at depth 3 : 604.811011388
test error at depth 4 : 526.53048095
test error at depth 5 : 463.571501548
test error at depth 6 : 348.338174181
test error at depth 7 : 228.428619434
test error at depth 8 : 107.664019556
test error at depth 9 : 10.1351299399
test error at depth 10 : 0.994190527863
[722.0611789853202, 677.8901216887728, 604.8110113875669, 526.5304809500908, 463.57150154818726, 348.33817418080304, 228.42861943390577, 107.66401955584888, 10.135129939896686, 0.9941905278626019]


In [15]:
# data inspection for the nodes of the trained tree

def feature_value_extract( valueSet):
    
    tmpval=1
    tmp_valSet= valueSet
    valSet_list=[]
    cnt=0
    
    while tmpval <= tmp_valSet:
        if tmpval & tmp_valSet !=0:
            valSet_list.append(cnt)
        tmpval=tmpval<<1
        cnt=cnt+1
        
    return valSet_list

def dfs_tree(tree, current_nodeIdx, current_featureVal_set, current_depth):
    
    nodeNum=len(tree)   
    if nodeNum == 0:
        return
    
    split_feature= tree[current_nodeIdx][0]
    split_valueSet= tree[current_nodeIdx][1]
        
    if split_feature==-1:
        print current_featureVal_set
        return;
    
    featureValue_list = feature_value_extract( split_valueSet)
    
    left_featureValue_set=[]
    right_featureValue_set=[]
    featureNum= len(current_featureVal_set)
    
    for i in range(0, featureNum):
        if i != split_feature:
            left_featureValue_set.append(  copy.deepcopy(current_featureVal_set[i])   )
            right_featureValue_set.append(  copy.deepcopy(current_featureVal_set[i])   )
        else:
            left_featureValue_set.append(  copy.deepcopy(featureValue_list )   )
            right_featureValue_set.append(copy.deepcopy(list(set(current_featureVal_set[i] )-set(featureValue_list)))   )
            
        
    traverse_tree(tree, current_nodeIdx*2+1,left_featureValue_set,current_depth+1)
    traverse_tree(tree, current_nodeIdx*2+2,right_featureValue_set, current_depth+1)
    
    return 

def training_data_summary( train_rdd ):

#     always remove outliers before
    train_rdd_clean= train_rdd.filter(lambda line: line[0]<1000)
    distinct_featureVals = train_rdd_clean.map(lambda line: (line[1], line[2],line[3],line[4]) ).distinct().collect()
   
    featureConfig_val={}
    for i in distinct_featureVals:
        
        tmprdd=train_rdd_clean.filter(lambda line:(line[1], line[2],line[3],line[4]) ==i ).map(lambda line: line[0])
        tmpval=tmprdd.reduce(lambda a,b:a+b)
        tmpval= tmpval/tmprdd.count()*1.0
        
        featureConfig_val.update( {i:tmpval} )
    
#     print featureConfig_val
    return featureConfig_val
        

def config_check( featureVal_set, feature_num):
    
    for i in range(0, feature_num):
        if len(featureVal_set[i]) >1 :
            return -1
    return 1

def lookup_featureConfig_val( configToVal_dict, value_set, feature_num):
    
    tmptuple=()
    for i in range(0, feature_num):
        tmptuple = tmptuple + ( value_set[i][0]  ,)
#     print '++++++++++++', tmptuple
    return configToVal_dict[tmptuple]
    
def test_error_mapFunc(line):
    tmpnode = search_nodeToData(line[1:numFeatures+1], test_tree_split)
    return (test_leaf_nodes[ tmpnode ]  - line[0])*(test_leaf_nodes[ tmpnode ]  - line[0])
    
def test_error_tree( current_depth, testRDD  ):
    
    test_cnt= testRDD.count()
    err_rdd=testRDD.map( lambda line: test_error_mapFunc(line))
    err_sum = err_rdd.reduce( lambda a,b: a+b)
    
    print 'test error at depth',current_depth, ':', err_sum*1.0/test_cnt
    return err_sum*1.0/test_cnt
    
def bfs_tree(tree, total_featureVal_set, depth, tree_esti, trainRDD ):
    
    nodeNum=len(tree)   
    if nodeNum == 0:
        return
    
    feature_ConfigToVal = training_data_summary( trainRDD )
    
    interNode_cnt=0
    non_leaf_cnt=0
    recog_config_cnt=0
    
    qu=[]
    current_depth=0
    qu.append((0,total_featureVal_set,0))
    
    
    recog_config_cnt_depth=[]
    recog_conf=[]
    
    while current_depth<= depth:
        
        print 'non_leaf_node count at previous depth', non_leaf_cnt
        
        print 'depth', current_depth,':', len(qu)
        tmpqu=[]
        
        non_leaf_cnt=0
        
        for i in qu:
            current_nodeIdx= i[0] 
            current_featureVal_set= i[1]
            
            split_feature= tree[current_nodeIdx][0]
            split_valueSet= tree[current_nodeIdx][1]
                 
            if split_feature==-1: 
                
                if config_check( current_featureVal_set, featureNum) ==1:
                    recog_config_cnt= recog_config_cnt+1
                    tmpval=lookup_featureConfig_val( feature_ConfigToVal, current_featureVal_set, featureNum)
                    
                    print current_featureVal_set, 'has the estimate:', tree_esti[ current_nodeIdx*2+1],\
                    'true:',tmpval
                    #result
                    recog_conf.append( (tree_esti[ current_nodeIdx*2+1], tmpval) )
                    
                else:
                    print current_featureVal_set, 'has the estimate:', tree_esti[ current_nodeIdx*2+1],'!!! not seperated !!!'
                
                interNode_cnt=interNode_cnt+1
                continue
                
            else:
                non_leaf_cnt=non_leaf_cnt+1
                print current_featureVal_set,'non-leaf node'
                
                
            featureValue_list = feature_value_extract( split_valueSet)
    
            left_featureValue_set=[]
            right_featureValue_set=[]
            featureNum= len(current_featureVal_set)
    
            for i in range(0, featureNum):
                if i != split_feature:
                    left_featureValue_set.append(  copy.deepcopy(current_featureVal_set[i])   )
                    right_featureValue_set.append(  copy.deepcopy(current_featureVal_set[i])   )
                else:
                    left_featureValue_set.append(  copy.deepcopy(featureValue_list )   )
                    right_featureValue_set.append(copy.deepcopy(list(set(current_featureVal_set[i] )-set(featureValue_list)))   )
            
            tmpqu.append(  (current_nodeIdx*2+1,left_featureValue_set, current_nodeIdx)  )
            tmpqu.append(  (current_nodeIdx*2+2,right_featureValue_set,current_nodeIdx) )
        
        qu=copy.deepcopy(tmpqu)
        current_depth= current_depth+1
        
        #result
        recog_config_cnt_depth.append( recog_config_cnt )
    
    
# print out leaf nodes
    print 'bottom depth', current_depth,':', len(qu)
    
    for i in qu:
        current_nodeIdx= i[0] 
        current_featureVal_set= i[1]
        parent_nodeIdx= i[2]
        
        if 2*parent_nodeIdx+1 == current_nodeIdx: 
#         parent_nodeIdx == (current_nodeIdx*2+1):
            tree_esti_val= tree_esti[ parent_nodeIdx*2]
        else:
            tree_esti_val= tree_esti[ parent_nodeIdx*2+1] 
        
        if config_check( current_featureVal_set, featureNum) ==1:
            recog_config_cnt= recog_config_cnt+1
            tmpval=lookup_featureConfig_val( feature_ConfigToVal, current_featureVal_set, featureNum)
            
            #result
            recog_conf.append( (tree_esti_val, tmpval) )
            
            print current_featureVal_set, 'has the estimate:', tree_esti_val,\
                    'true:',tmpval
        else:
            print current_featureVal_set, 'has the estimate:', tree_esti_val
    
    #result
    recog_config_cnt_depth.append( recog_config_cnt )
        
    print 'inter-node number:', interNode_cnt
    print ' *******  number of identified configurations:', recog_config_cnt
    
    tmpval=0
    for i in range(0,recog_config_cnt):
        tmpval=tmpval+ (recog_conf[i][0]-recog_conf[i][1])*(recog_conf[i][0]-recog_conf[i][1])
    
    return (recog_conf, recog_config_cnt_depth, tmpval*1.0/recog_config_cnt)

# len( tree_history_split)
# len(tree_history_esti)


In [ ]:
#do not run this part
# training and testing process

# without outliers
tmpdta = dta.filter(lambda line: line[0]<1000).takeSample(False, 100000, 12243)
dta_train = sc.parallelize(tmpdta).cache().sample(False, .7, 12345)
dta_test = sc.parallelize(tmpdta).cache().sample(False, .3, 43243)

maxdepth=10
numFeatures=3

node_split=[]
node_test=[]
run_time=[]

#------------ estimate phase ---------------------
# def search_nodeToData(features, tree):
#     nodeNum=len(tree)   
#     if nodeNum == 0:
#         return 0;
    
#     current_nodeIdx=0
#     while current_nodeIdx< nodeNum:
#         split_feature= tree[current_nodeIdx][0]
#         split_valueSet= tree[current_nodeIdx][1]
        
#         if split_feature==-1:
#             return -1
#         if (( 1<<features[split_feature]) &  split_valueSet) != 0 :
#             current_nodeIdx=current_nodeIdx*2+1
#         else:
#             current_nodeIdx=current_nodeIdx*2+2        
#     return current_nodeIdx - nodeNum

def tree_test_mapFunc(line):
    tmpnode = search_nodeToData(line[1:numFeatures+1], node_split)
    return (leaf_nodes[ tmpnode ]  - line[0])*(leaf_nodes[ tmpnode ]  - line[0])
    
def tree_test( testData_rdd ):
    err_rdd=testData_rdd.map( lambda line: tree_test_mapFunc(line))
    err_sum = err_rdd.reduce( lambda a,b: a+b)
    return err_sum / testData_rdd.count()
#------------------------------------------------
    
#ini parameter on the cluster
currentNode_split_fromMaster = sc.broadcast([])
test_err = []
train_err = []
#prepare feature value set
feature_valueSet = data_featureValues_collect( dta_train )
feature_valueList=[]
for i in range(0, len(feature_valueSet)):
    feature_valueList.append( ( i, list(feature_valueSet[i][1]) ) )
    
    
for i in range(6,maxdepth):
    
    start = time.time() 
    
    del node_split[:]
    del node_test[:]
    currentNode_split_fromMaster = sc.broadcast([])
    
    for cur_depth in range(0,i):
        
        current_nodeNum= int( math.pow(2,  cur_depth)) 
        
        dataToNode = dataToNode_assignment( dta_train )
        find_bestSplit_exact(dataToNode,current_nodeNum, node_split, node_test, feature_valueList)   
#         ( dataToNode, current_NumNodes, nodes_tree, nodes_tree_test, feature_valueSet):
        #for whole tree
        currentNode_split_fromMaster = sc.broadcast(node_split)
    
    end = time.time()
    elapsed = end - start    
    leaf_nodes = node_test[ len(node_test)-  (int)( math.pow(2,i)): len(node_test)  ]

    
    tmp_test_err= tree_test( dta_test )
    tmp_train_err= tree_test( dta_train )
    
    test_err.append( tmp_test_err)
    train_err.append( tmp_train_err )
    run_time.append(elapsed)
    
    print "error at tree height", i,":",  tmp_test_err, tmp_train_err
    print "running time at tree height", i,":",  elapsed
    

In [ ]:

error at tree height 1 : 836.120673266 993197.408799
running time at tree height 1 : 71.6192398071
error at tree height 2 : 805.794778995 994017.544055
running time at tree height 2 : 138.060220957
error at tree height 3 : 765.500220461 994394.511005
running time at tree height 3 : 208.832412004
    
error at tree height 4 : 734.829314704 993707.63729
running time at tree height 4 : 281.407808065
    
error at tree height 5 : 644.236932264 994344.82116
running time at tree height 5 : 4879.72735

error at tree height 6 : 529.4375226396904 994434.7416043472

error at tree height 7 : 770.5749206468843 992877.3588177576
    

In [ ]:
# singal-run training  process  
dta_train = dta.sample(False, .7, 12345)
dta_test = dta.sample(False, .3, 43243)
dta_test = dta_test.filter(lambda line: line[0] <1000 )

#parameters
maxdepth=
numFeatures=3

node_split=[]
node_test=[]
#ini parameter
currentNode_split_fromMaster = sc.broadcast([])

#prepare feature value set
feature_valueSet = data_featureValues_collect( dta_train )
feature_valueList=[]
for i in range(0, len(feature_valueSet)):
    feature_valueList.append( ( i, list(feature_valueSet[i][1]) ) )
    

for i in range(0,maxdepth):
    
    start = time.time()
    current_nodeNum= int( math.pow(2,  i))
    
    print 'current split decision:',currentNode_split_fromMaster.value

    dataToNode = dataToNode_assignment( dta_train )
    cluster_end = time.time() 
    find_bestSplit_exact(dataToNode,current_nodeNum,node_split,node_test, feature_valueList)   
#   ( dataToNode, current_NumNodes, nodes_tree, nodes_tree_test, feature_valueSet):
    currentNode_split_fromMaster = sc.broadcast(node_split)
  
    end = time.time() 

    print i,'-th level running time: ', cluster_end - start,'sec', end- cluster_end, 'sec'
    
    

In [118]:
#select one feature-value
feature_id=1
feature_val=11

dataToFeatureValue_rdd=node_data.filter(lambda line:line[1][feature_id+1]==feature_val).map(lambda line:line[1][0])

dataToFeatureValue_rdd_count = dataToFeatureValue_rdd.count()
        
# med_tmpLarge= dataToFeatureValue_rdd.top( dataToFeatureValue_rdd_count/2+1  )
med_tmp = dataToFeatureValue_rdd.top( dataToFeatureValue_rdd_count/2  )
med_tmp.sort()
tmpcnt= len(med_tmp)
print med_tmp[ tmpcnt-1]


69015

In [135]:
#select several feature-values
feature_values=[1,2,3]

dataToFeatureValue_rdd=node_data.filter(lambda line:line[1][feature_id+1] in feature_values).map(lambda line:line[1][0])

dataToFeatureValue_rdd_count = dataToFeatureValue_rdd.count()

print dataToFeatureValue_rdd_count

437547
